# Исследование надежности заемщиков.

### ОГЛАВЛЕНИЕ

#### Предисловие 

#### Описание данных

#### Шаг 1. Импортируем данные, изучим и подготовим их к анализу¶

#### Шаг 2. Заполнение пропусков

#### Шаг 2.1 Проверка данных на аномалии и исправления

#### Шаг 2.2. Удаление дубликатов

#### Шаг 2.3. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма

#### Шаг 2.4. Категоризация дохода

#### Шаг 2.5. Категоризация целей кредита

#### Ответы на вопросы

#### Общий вывод

## Предисловие

 **Заказчик** - кредитный отдел банка

**Цель исследования**:
    

1) Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок

**Результаты исследования** - будут учтены при построении модели кредитного скоринга 
(специальной системы, которая оценивает способность потенциального заемщика вернуть кредит банку)

### Описание данных

Изучим документацию к данным. Согласно документации:

* `children` - количество детей в семье

* `days_employed` - общий трудовой стаж в днях

* `dob_years` - возраст клиента в годах

* `education` - уровень образования клиента

* `education_id` - идентификатор уровня образования

* `family_status` - семейное положение

* `family_status_id` - идентификатор семейного положения

* `gender` - пол клиента

* `income_type` - тип занятости

* `debt` - имел ли задолженность по возврату кредитов

* `total_income` - ежемесячный доход

* `purpose` - цель получения кредита

### Шаг 1. Импортируем данные, изучим и подготовим их к анализу

In [1]:
# импортируем библиотеку pandas
import pandas as pd

In [2]:
# читаем файл с данными и сохраняем его в df
df = pd.read_csv('D:/Project/2/data.csv')

Выведем на экран первые десять строк таблицы:

In [3]:
# получаем первые 10 строк таблицы df
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Получим общую информацию о таблице:

In [4]:
# получаем общую информацию о данных в таблице df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице 12 столбцов. Тип данных - строки (`object`), целые числа (`int64`), вещественные числа (`float64`).

Проверим написание столбцов, с целью приведения их к единому регистру:

In [5]:
# Выведем на экран название столбцов
df.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

Как видим, столбы написаны в едином регистре, в правке не нуждаются

Определим, есть ли в таблице пропущенные значения. Для этого используем метод `isna()`

In [6]:
# подсчитаем количество пропущенных значений 
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Как видим, столбцы `days_employed` и `total_income` имеют пропущенные значения

Изучим их более внимательно:

In [7]:
# подсчитаем, совпадает ли количество пропущенных значений столбцов `days_employed` и `total_income`
df[(df['days_employed'].isna() == True) & df['total_income'].isna() == True].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          2174 non-null   int64  
 1   days_employed     0 non-null      float64
 2   dob_years         2174 non-null   int64  
 3   education         2174 non-null   object 
 4   education_id      2174 non-null   int64  
 5   family_status     2174 non-null   object 
 6   family_status_id  2174 non-null   int64  
 7   gender            2174 non-null   object 
 8   income_type       2174 non-null   object 
 9   debt              2174 non-null   int64  
 10  total_income      0 non-null      float64
 11  purpose           2174 non-null   object 
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


Как видим, пропущенные значения в столбце `days_employed` и `total_income` совпадают

Посчитаем, какую долю от общего количества составляют пропущенные значения:

In [8]:
# Разделим пропущенные значения на общее количество значений. Результат умножим на 100% и округлим до двух знаков после запятой
total = 21525
missing_value = 2174
precent_total = (missing_value / total)
display(f'Доля пропущенных значений от общего количества составляет: {precent_total:.2%}')

'Доля пропущенных значений от общего количества составляет: 10.10%'

**10,10%** - составляет довольно большую долю пропущенных значений от общего количества. 
Возможные причины их появления - люди не имеют официального места работы и официального заработка либо не указывают место работы и заработок. Также возможна ошибка при заведении данных в таблицу. Удалить их нельзя, так как это повлияет на вывод исследования. Придется их заменить.

### Шаг 2. Заполнение пропусков

Один из способов замены - замена пропусков медианным значением. Так как `days_employed` (общий трудовой стаж) 
и `total_income` (ежемесячный доход) количественные переменные, то пропуски в таких значениях можно заполнить 
характерными значениями.

При изучении данных в таблице видны аномалии: данные в столбцах имеют отрицательные значение.
Возможно, была допущена ошибка при заведении данных в таблицу. Необходимо поменять отрицательные значения на 
положительные, подсчитаем среднюю арифметическую и медиану.

Отрицательные значения на положительные в столбцах `days_employed` (общий трудовой стаж) 
и `total_income` (ежемесячный доход) поменяем с помощью медианы (метод `abs()`)

In [9]:
df['days_employed'] = df['days_employed'].abs()

In [10]:
df['total_income'] = df['total_income'].abs()

Чтобы определить медиану, нужно внимательно рассмотреть таблицу `total_incomе` (ежемесячный доход). 
Произведем группировку данных в нашей таблице.

In [11]:
df_income_type = df.groupby(['income_type']).agg({'total_income': ['sum','count']})

In [12]:
display(df_income_type)

total_income       
                          sum  count
income_type                         
безработный      2.626795e+05      2
в декрете        5.382913e+04      1
госслужащий      2.242186e+08   1312
компаньон        9.264647e+08   4577
пенсионер        4.721299e+08   3443
предприниматель  4.991631e+05      1
сотрудник        1.616062e+09  10014
студент          9.820163e+04      1

Найдем медианное значение и заполним пропуски по конкретному типу занятости

In [13]:
temp = df.copy()

In [14]:
median_sot = df.loc[df['income_type'] == 'сотрудник', 'total_income'].median()
median_gos = df.loc[df['income_type'] == 'госслужащий', 'total_income'].median()
median_com = df.loc[df['income_type'] == 'компаньон', 'total_income'].median()
median_pen = df.loc[df['income_type'] == 'пенсионер', 'total_income'].median()
median_pred = df.loc[df['income_type'] == 'предприниматель', 'total_income'].median()

In [15]:
df['total_income'] = df['total_income'].fillna(value = median_sot)
df['total_income'] = df['total_income'].fillna(value = median_gos)
df['total_income'] = df['total_income'].fillna(value = median_com)
df['total_income'] = df['total_income'].fillna(value = median_pen)
df['total_income'] = df['total_income'].fillna(value = median_pred)

In [16]:
# Код ревьюера
temp.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [17]:
# Код ревьюера
temp.loc[temp['income_type'] == 'сотрудник', 'total_income'] = \
                                temp.loc[temp['income_type'] == 'сотрудник', 'total_income'].fillna(value = median_sot)

In [18]:
# Код ревьюера
temp.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        1069
purpose                0
dtype: int64

In [19]:
median_days_sot = df.loc[df['income_type'] == 'сотрудник', 'days_employed'].median()
median_days_gos = df.loc[df['income_type'] == 'госслужащий', 'days_employed'].median()
median_days_com = df.loc[df['income_type'] == 'компаньон', 'days_employed'].median()
median_days_pen = df.loc[df['income_type'] == 'пенсионер', 'days_employed'].median()
median_days_pred = df.loc[df['income_type'] == 'предприниматель', 'days_employed'].median()

In [20]:
df['days_employed']= df['days_employed'].fillna(value = median_days_sot)
df['days_employed']= df['days_employed'].fillna(value = median_days_gos)
df['days_employed']= df['days_employed'].fillna(value = median_days_com)
df['days_employed']= df['days_employed'].fillna(value = median_days_pen)
df['days_employed']= df['days_employed'].fillna(value = median_days_pred)

In [21]:
# Проверим наличие пропусков в столбцах
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Пропуски заполнены медианным значением

### Шаг 2.1 Проверка данных на аномалии и исправления

Теперь проверим каждый столбец на аномалии (значения, которые не отражают действительность).
Если они есть, то исправим их

#### Столбец `children` - Дети

Рассмотрим первый столбец - Дети. Применим метод `.value_counts()`, который возвращает уникальные значения 
и количество их упоминаний

In [22]:
# Применяем метод `.value_counts()` к столбцу df['children']
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Как видим, в данном столбце встречаются аномалии: у 47 человек количество детей `-1`, а у 76 человек `20` детей.
Думаю, что ошибка возникла случайно, вследствие человеческого фактора. 
Исправим `-1` на `1`, а `20` заменим медианным значением

In [23]:
# Для замены отрицательного значения на положительное воспользуемся методом `.replace()`
df['children'] = df['children'].replace(-1, 1)

In [24]:
# Для замены числа `20` на медианное значение сделаем логическую индексацию и воспользуемся методом `.median()`
median_children = df.loc[df.loc[:, 'children'] != 20]['children'].median()
df['children'] = df['children'].replace(20, median_children)

Проверим результат:

In [25]:
df['children'].value_counts()

0    14225
1     4865
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

Видим, что аномалии отсутствуют

#### Столбец `days_employed` - общий трудовой стаж в днях

In [26]:
# Посмотрим уникальные значения столбца
df['days_employed'].value_counts()

1574.202821      2174
8437.673028         1
5135.928528         1
354500.415854       1
769.717438          1
                 ... 
1099.957609         1
209.984794          1
398099.392433       1
1271.038880         1
1984.507589         1
Name: days_employed, Length: 19352, dtype: int64

Отрицательные значения в данном столбце убрали, пропуски заполнили медианным значением

Проверим тип данных:

In [27]:
df['days_employed']

0          8437.673028
1          4024.803754
2          5623.422610
3          4124.747207
4        340266.072047
             ...      
21520      4529.316663
21521    343937.404131
21522      2113.346888
21523      3112.481705
21524      1984.507589
Name: days_employed, Length: 21525, dtype: float64

Видим, что тип данных в столбце - `float64`, что означает вещественные числа.
Заменим на целые числа:

In [28]:
# Для замены типа данных воспользуемся методом `.astype()`
df['days_employed'] = df['days_employed'].astype('int')

In [29]:
df['days_employed']

0          8437
1          4024
2          5623
3          4124
4        340266
          ...  
21520      4529
21521    343937
21522      2113
21523      3112
21524      1984
Name: days_employed, Length: 21525, dtype: int32

Типа данных поменяли на целые числа

#### Столбец `dob_years` - возраст клиента в годах

In [30]:
df['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
22    183
66    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

Как видим, данные тоже содержат аномалии: 101 человек имеет возраст 0. 
Так как 101 человек от общего количества людей составляет `((101/21525) * 100)` примерно `0,5%`, 
то такое значение не будет влиять на результат

#### Столбец `education` - уровень образования клиента

In [31]:
df['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

В данном столбце присутствуют аномалии: разный регистр букв. Приведем буквы в словах к одному регистру,
для этого воспользуемся методом `.str.lower()`

In [32]:
# Приводим буквы к одному регистру
df['education'] = df['education'].str.lower()
df['education'].value_counts()


среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Как видим, аномалия устранена, буквы имеют единый регистр

#### Столбец `education_id` - идентификатор уровня образования

In [33]:
df['education_id'].value_counts()

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

Столбец корректен. Как видим, данные соответствут уровню образования клиента

#### Столбец `family_status` - семейное положение

In [34]:
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

В столбце присутствуют аномалии. Знакомым нам методом `.str.lower()` приведем буквы в словах к одному регистру:

In [35]:
# Приводим буквы к одному регистру
df['family_status'] = df['family_status'].str.lower()
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Аномалия устранена, буквы имеют единый регистр

#### Столбец `family_status_id` - идентификатор семейного положения

In [36]:
df['family_status_id'].value_counts()

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

Столбец корректен.

#### Столбец `gender` - пол клиента

In [37]:
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

В данном столбце имеется аномальное значение `XNA`. Оставим это значение, так как оно на результат исследования 
не будет влиять

#### Столбец `income_type` - тип занятости

In [38]:
df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

В данном столбце данные в правке не нуждаются

#### Столбец `debt` - имел ли задолженность по возврату кредитов

In [39]:
df['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

В данном столбце данные в правке не нуждаются

#### Столбец `total_income` - ежемесячный доход

In [40]:
df['total_income']

0        253875.639453
1        112080.014102
2        145885.952297
3        267628.550329
4        158616.077870
             ...      
21520    224791.862382
21521    155999.806512
21522     89672.561153
21523    244093.050500
21524     82047.418899
Name: total_income, Length: 21525, dtype: float64

Данный столбец нами уже исправлен: исправлены отрицательные значения, удалены пропуски.
Однако, тип данного столбца - `float64` (вещественные числа). Поменяем его на целочисленный

In [41]:
# Для замены типа стоблца воспользуемся методом `astype()`
df['total_income'] = df['total_income'].astype('int')

In [42]:
# Проверяем результат
df['total_income']

0        253875
1        112080
2        145885
3        267628
4        158616
          ...  
21520    224791
21521    155999
21522     89672
21523    244093
21524     82047
Name: total_income, Length: 21525, dtype: int32

Тип столбца `ежемесячный доход` поменяли с вещественного числа на целочисленный

#### Столбец `purpose` - цель получения кредита

In [43]:
df['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

В данном столбце имеются неявные дубликаты. В дальнейшем придется их удалить

### Шаг 2.2. Удаление дубликатов

Осуществим поиск дубликатов. Для этого воспользуемся функцией `.duplicated()`

In [44]:
# Подсчитаем количество дубликатов
df.duplicated().sum()

71

In [45]:
# Удалим явные дубликаты (с удалением старых индексов и формированием новых)
df = df.drop_duplicates().reset_index()

In [46]:
# Проверим результат: подсчитаем количество дубликатов
df.duplicated().sum()

0

Дубликаты удалены

### Шаг 2.3. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма

Создадим два новых датафрейма: первый со столбцами `education` и `education_id`;
второй со столбцами `family_status_id` и `family_status`

In [47]:
first_new_df = df[['education', 'education_id']]
second_new_df = df[['family_status_id', 'family_status']]

In [48]:
# Выведем на экран с целью проверки
display(first_new_df)

,education,education_id
0,высшее,0
1,среднее,1
2,среднее,1
3,среднее,1
4,среднее,1
...,...,...
21449,среднее,1
21450,среднее,1
21451,среднее,1
21452,среднее,1


In [49]:
# Выведем на экран с целью проверки
display(second_new_df)

,family_status_id,family_status
0,0,женат / замужем
1,0,женат / замужем
2,0,женат / замужем
3,0,женат / замужем
4,1,гражданский брак
...,...,...
21449,1,гражданский брак
21450,0,женат / замужем
21451,1,гражданский брак
21452,0,женат / замужем


Проверим наличие дубликатов и если есть - то удалим их:

In [51]:
first_new_df.duplicated().sum()

21449

In [52]:
second_new_df.duplicated().sum()

21449

In [53]:
first_new_df = first_new_df.drop_duplicates().reset_index()

In [54]:
second_new_df = second_new_df.drop_duplicates().reset_index()

In [55]:
display(first_new_df)

,index,education,education_id
0,0,высшее,0
1,1,среднее,1
2,13,неоконченное высшее,2
3,31,начальное,3
4,2962,ученая степень,4


In [56]:
display(second_new_df)

,index,family_status_id,family_status
0,0,0,женат / замужем
1,4,1,гражданский брак
2,18,2,вдовец / вдова
3,19,3,в разводе
4,24,4,не женат / не замужем


Удалим из исходного датафрейма столбцы education и family_status:

In [57]:
#  Воспользуемся методом `.drop()`, а `axis=1` указывает, что удалять нужно столбец, а не строку
df.drop(['education', 'family_status'], axis=1)

,index,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...
21449,21520,1,4529,43,1,1,F,компаньон,0,224791,операции с жильем
21450,21521,0,343937,67,1,0,F,пенсионер,0,155999,сделка с автомобилем
21451,21522,1,2113,38,1,1,M,сотрудник,1,89672,недвижимость
21452,21523,3,3112,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля


### Шаг 2.4. Категоризация дохода

На основании диапазонов назначим кредитополучателю категорию, в зависимости от ежемесячного дохода, от `A` до `E`

In [58]:
# Создадим функцию: на вход подадим доход клиента, а возвращает категорию клиента
def total(total_income):
    if 0 <= total_income <= 30000:
        return 'E'
    if 30001 <= total_income <= 50000:
        return 'D'
    if 50001 <= total_income <= 200000:
        return 'C'
    if 200001 <= total_income <= 1000000:
        return 'B'
    return 'A'

In [59]:
# Создадим отдельный столбец с категориями и в ячейке запишем значения, возвращаемые функцией.
# Для этого воспользуемся методом `.apply()`
df['total_income_category'] = df['total_income'].apply(total)

In [60]:
df.head()

,index,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B
1,1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C
3,3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B
4,4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C


### Шаг 2.5. Категоризация целей кредита

Создадим функцию, которая на основании данных и столбца `purpose` сформирует новый столбец для категоризации цели кредита.
Затем воспользуемся методом `.apply()`

In [61]:
def total(purpose):
    if 'авто' in purpose:
        return 'операции с автомобилем'
    elif 'свад' in purpose:
        return 'проведение свадьбы'
    elif 'образ' in purpose:
        return 'получение образования'
    else:
        return 'операции с недвижимостью'

In [62]:
df['purpose_category'] = df['purpose'].apply(total)

In [63]:
df.head(10)

,index,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,C,получение образования
8,8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


## Ответы на вопросы

#### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

Осуществим группировку данных: в столбей `количество детей`, передадим словарь 
методом `agg()`, где ключ - это название слолбца `имел ли задолженность по возврату кредита`

In [64]:
df_children_debt = df.groupby(['children']).agg({'debt': ['sum','count']})

In [65]:
df_children_debt

debt       
           sum  count
children             
0         1071  14167
1          445   4855
2          194   2052
3           27    330
4            4     41
5            0      9

Далее, вычислим процент клиентов с детьми,допустившими просрок по кредиту:

In [66]:
precent_total_debt = (df_children_debt['debt']['sum'] / df_children_debt['debt']['count']) * 100

In [67]:
precent_total_debt

children
0    7.559822
1    9.165808
2    9.454191
3    8.181818
4    9.756098
5    0.000000
dtype: float64

##### Вывод 1:

Как видим из результатов, количество детей влияет на процент клиентов, допустившим просрок по кредиту:
у клиентов с 5 детьми просроков не было. Скорее всего, это связано с тем, что такие клиенты более отвественно подходят к решению о кредите, но скорее всего, не берут его совсем. 
Меньший процент просрочки по кредитам у клиентов, имеющих 1 ребенка. 
Таким образом, можно сделать вывод, что чем меньше детей у клиента, тем реже они допускают просроки по кредитам

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [68]:
df_family_debt = df.groupby(['family_status']).agg({'debt': ['sum','count']})

In [69]:
df_family_debt

debt       
                       sum  count
family_status                    
в разводе               85   1195
вдовец / вдова          63    959
гражданский брак       388   4151
женат / замужем        931  12339
не женат / не замужем  274   2810

In [70]:
precent_total_debt_f = (df_family_debt['debt']['sum'] / df_family_debt['debt']['count']) * 100

In [71]:
precent_total_debt_f

family_status
в разводе                7.112971
вдовец / вдова           6.569343
гражданский брак         9.347145
женат / замужем          7.545182
не женат / не замужем    9.750890
dtype: float64

##### Вывод 2: 
Зависимость между семейным положением и возвратом кредита в срок прослеживается: 
меньше всего задолженность у категории вдовец/вдова. Скорее всего, это связано с тем, что при оформлении кредита 
жизнь человека страхуется и после его смерти страховая компания платит банку определенный процент.
Также небольшая разница у тех, кто женат/замужем или в разводе. Самый большой процент просроков у клиентов, 
состоящих в гражданском браке либо не женатых/не замужем. Скорее всего, это связано с тем, что люди не ответственно подходят
к возвращению кредита


##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [72]:
df_income = df.groupby(['total_income_category']).agg({'debt':['sum','count']})

In [73]:
df_income

debt       
                        sum  count
total_income_category             
A                         2     25
B                       356   5041
C                      1360  16016
D                        21    350
E                         2     22

`A` - очень высокий уровень дохода
`В` - высокий уровень дохода
`С` - выше среднего уровень дохода
`D` - средний уровень дохода
`Е` - уровень дохода ниже среднего

In [74]:
precent_total_debt_in = (df_income['debt']['sum'] / df_income['debt']['count']) * 100

In [75]:
precent_total_debt_in

total_income_category
A    8.000000
B    7.062091
C    8.491508
D    6.000000
E    9.090909
dtype: float64

##### Вывод 3: 
Зависимость между уровнем дохода и возвратом кредита в срок прослеживается: 
меньше всего просрочек допускают клиенты со среднем уровнем дохода. 
Ну и что закономерно: больше всего просрочек по кредиту допускают клиенты с уровнем дохода ниже среднего

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

In [80]:
# По умолчанию, если не указывать параметр `aggfunc`, то метод `pivot_table`
# рассчитает среднее арифметическое значение, указанное в параметре `values`
pd.pivot_table(df,
              index=['purpose_category'],
              values='debt')              

,debt
purpose_category,
операции с автомобилем,0.093590
операции с недвижимостью,0.072334
получение образования,0.092200
проведение свадьбы,0.080034


##### Вывод 4: 

Разные цели кредита несущественно, но все же влияют на возврат кредита: меньше всего клиентов не оплачивают кредит по операции с недвижимостью, что закономерно, т.к. зачастую жилье находится в залоге у банка. Больше всего просрочек по кредиту допускают по операциям с автомобилем и получением образования

## Общий вывод:

В своей работе мы исследовали надежность заемщиков. Выяснили, что самые частые цели кредита - это операции с автомобилем, операции с недвижимостью, получение образованиия и свадьба. Меньше всего клиентов допускают просроченные платежи по кредиту за недвижимость. 
По уровню дохода клиентов, меньше всего просроченных платежей у клиентов со средним уровнем дохода. Самые ненадежные клиенты - у которых уровень дохода ниже среднего.
Анализируя семейное положение клиентов, приходим к выводу, что вдовец (вдвова) реже допускают просроченные платежи по кредитам. Не женатые (не замужние) и клиенты, живущие в гражданском браке допускают просроченных платежей больше всего.
При изучении количества детей на вовзрат кредита в срок, приходим к выводу, что чем меньше детей, тем клиент реже допускает просроченных платежей по кредиту. Правда, в нашем исследовании мы выяснили, что клиенты, у кого 5 детей совсем не допускают просроченных платежей по кредиту. Но это, скорее всего, связано с тем, что люди более ответственно подходят к решению о кредите, либо не берут по причине невозможности вернуть его в срок.